## Libs

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
from scipy import stats
import re
import time
import matplotlib.pyplot as plt

## Pre Processing

In [ ]:
movie = pd.read_html('https://en.wikipedia.org/wiki/List_of_films_based_on_video_games')[0]

In [ ]:
movie = movie[['Title','Rotten Tomatoes']]
movie_filter = (movie['Rotten Tomatoes'] != 'TBA')

movie = movie[movie_filter]
movie = movie[:-1]

In [ ]:
rotten_ref = movie['Rotten Tomatoes'].str.split("[", n=1,expand=True)[1]
rotten_ref = rotten_ref.str.split("]", n=1,expand=True)[0]
rotten_ref


## Web Scraping

In [ ]:
link_array = []
critc_array = []
audi_array = []


n = 0

driver = webdriver.Chrome()


for ref in rotten_ref:
    
    try:

        driver.get('https://en.wikipedia.org/wiki/List_of_films_based_on_video_games')
        xpath_link = '/html/body/div[3]/div[3]/div[5]/div[1]/div[6]/div/ol/li[{}]/span[2]/cite/a[1]'.format(ref)
        tomatoes_link = driver.find_element_by_xpath(xpath_link)
        time.sleep(3)
        tomatoes_link.click()

        rotte_info = driver.find_element_by_xpath('/html/body/div[5]/div[3]/section/div[2]/div[2]/score-board').text
        crict_note = rotte_info.split('\n')[2]
        audi_note = rotte_info.split('\n')[5]

        critc_array.append(crict_note)
        audi_array.append(audi_note)

        driver.get('https://en.wikipedia.org/wiki/List_of_films_based_on_video_games')

    except:

        critc_array.append('Nan')
        audi_array.append('Nan')
        driver.get('https://en.wikipedia.org/wiki/List_of_films_based_on_video_games')
    
    pass







## Final Touches

In [ ]:
print(critc_array, audi_array)


In [ ]:
movie['Critics Rating %'] = critc_array
movie['Audience Rating %'] = audi_array

movie = movie.drop('Rotten Tomatoes',axis=1)

movie['Critics Rating %'] = movie['Critics Rating %'].str.replace('%','')
movie['Audience Rating %'] = movie['Audience Rating %'].str.replace('%','')

movie['Critics Rating %'] = pd.to_numeric(movie['Critics Rating %'], errors='coerce')
movie['Audience Rating %'] = pd.to_numeric(movie['Audience Rating %'], errors='coerce')

In [ ]:
x = movie['Critics Rating']
y = movie['Audience Rating']


In [ ]:
movie.to_excel('movie_data.xlsx')

## Game Notes

In [156]:
names = pd.read_excel('movie_data.xlsx')
names = names['Title']
names = names.to_list(

In [158]:
driver = webdriver.Chrome()
driver.get('https://www.metacritic.com/')

title_game_array = []
rating_game_array = []
reference_array = []

search_box = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/header/nav/nav[2]/div[1]/div[2]/div/form/input[1]')

for name in names:

    search_box.click()
    search_box.send_keys(name)
    search_box.send_keys(Keys.ENTER)
    
    time.sleep(5)

    game_button = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div[3]/a/span[1]')
    game_button.click()

    time.sleep(5)

    score_button = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div[4]/div[1]/div[3]/a')
    score_button.click()

    search_box = driver.find_element_by_xpath('/html/body/div[2]/div/header/nav/nav[2]/div[1]/div[2]/div/form/input[1]')

    for i in range(1,11):

        try:
            
            game_rating_path = '/html/body/div[2]/div/div/div/div[3]/div/ul/li[{}]/div/div[2]/div/span'.format(i)
            game_name_path = '/html/body/div[2]/div/div/div/div[3]/div/ul/li[{}]/div/div[2]/div/h3/a'.format(i)
            game_rating = driver.find_element_by_xpath(game_rating_path).text
            game_name = driver.find_element_by_xpath(game_name_path).text

            title_game_array.append(game_name)
            rating_game_array.append(game_rating)
            reference_array.append(name)

            aux = 'Reference: {} -------- Name: {} ---- Rating : {}'.format(name, game_name, game_rating)
            print(aux)

        except:

            print('Not Found')
            title_game_array.append('Nan')
            rating_game_array.append('Nan')
            reference_array.append(name)

        pass

driver.close()

Reference: Detective Pikachu -------- Name: Detective Pikachu ---- Rating : 71
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Not Found
Reference: Sonic the Hedgehog -------- Name: 3D Sonic the Hedgehog 2 ---- Rating : 87
Reference: Sonic the Hedgehog -------- Name: Sega Ages: Sonic the Hedgehog 2 ---- Rating : 83
Reference: Sonic the Hedgehog -------- Name: Sonic the Hedgehog 2 ---- Rating : 82
Reference: Sonic the Hedgehog -------- Name: Sonic the Hedgehog 4: Episode I ---- Rating : 81
Reference: Sonic the Hedgehog -------- Name: 3D Sonic the Hedgehog ---- Rating : 81
Reference: Sonic the Hedgehog -------- Name: Sonic the Hedgehog 3 ---- Rating : 79
Reference: Sonic the Hedgehog -------- Name: Sonic the Hedgehog (Live Arcade) ---- Rating : 77
Reference: Sonic the Hedgehog -------- Name: Sega Ages: Sonic the Hedgehog ---- Rating : 77
Reference: Sonic the Hedgehog -------- Name: Sonic the Hedgehog 4: Episode I ---- Rating : 74
Reference: Sonic the Hedge

In [159]:
meta = pd.DataFrame(title_game_array)
meta['Rating'] = rating_game_array
meta['Reference'] = reference_array

meta.to_excel('meta.xlsx')